Implementation with Sequential Method

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4 ,28, 28, 3
n_conv_neurons =[10,20,30]
n_dense_neurons = [50,30,10]
k_size, padding = 3,'same'
pool_size, pool_strides = 2,2

x=tf.random.normal(shape=(N, n_H, n_W, n_C))
print(x.shape)
# ----------- Feature Extractor --------------

model = Sequential()
model.add(Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding,
               activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides= pool_strides))
model.add(Conv2D(filters=n_conv_neurons[1], kernel_size = k_size, padding = padding,
               activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides= pool_strides))
model.add(Conv2D(filters=n_conv_neurons[2], kernel_size = k_size, padding = padding,
               activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides= pool_strides))
model.add(Flatten()) # dense layer에 벡터 형식으로 넣기 위해 마지막에 flatten()을 해준것

# ----------- Classifier --------------

model.add(Dense(units=n_dense_neurons[0], activation='relu'))
model.add(Dense(units=n_dense_neurons[1], activation='relu'))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

predictions = model(x)
print(predictions.shape) 

# 결과적으로 predictions를 구하는 방식이 될것이고, 4,10은 쉽게 설명해서
# 4명의 학생에 10개의 성적의 각각의 확률이 될 것이다.



(4, 28, 28, 3)
(4, 10)


In [ ]:
# for문을 통한 방식
model = Sequential()

for n_conv_neuron in  n_conv_neurons:
  model.add(Conv2D(filters=n_conv_neurons, kernel_size = k_size, padding = padding,
               activation='relu'))
  model.add(MaxPooling2D(pool_size=pool_size, strides= pool_strides))

model.add(Flatten()) 

for n_dense_neuron in  n_conv_neurons:
  model.add(Dense(units=n_dense_neurons, activation='relu'))
model.add(Dense(units=n_dense_neurons[-1], activation='softmax'))

Implementation with Model Sub-classing

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4 ,28, 28, 3
n_conv_neurons =[10,20,30]
n_dense_neurons = [50,30,10]
k_size, padding = 3,'same'
pool_size, pool_strides = 2,2

x=tf.random.normal(shape=(N, n_H, n_W, n_C))

class TestCNN(Model):
  def __init__(self):
    super(TestCNN,self).__init__()

    global n_conv_neurons,n_dense_neurons

    self.conv1=Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding,
               activation='relu')
    self.conv1_pool=MaxPooling2D(pool_size=pool_size, strides= pool_strides)

    self.conv2=Conv2D(filters=n_conv_neurons[1], kernel_size = k_size, padding = padding,
               activation='relu')
    self.conv2_pool=MaxPooling2D(pool_size=pool_size, strides= pool_strides)

    self.conv3=Conv2D(filters=n_conv_neurons[2], kernel_size = k_size, padding = padding,
               activation='relu')
    self.conv3_pool=MaxPooling2D(pool_size=pool_size, strides= pool_strides)

    self.flatten = Flatten()

    self.dense1= Dense(units=n_dense_neurons[0], activation='relu')
    self.dense2= Dense(units=n_dense_neurons[1], activation='relu')
    self.dense3= Dense(units=n_dense_neurons[2], activation='softmax')

  def call(self,x):
    x = self.conv1(x)
    x = self.conv1_pool(x)
    print(x.shape) # 이렇게 중간중간 어떤 형태를 띄는지 확인 할 수있는게 Model의 장점이다.
    x = self.conv2(x)
    x = self.conv2_pool(x)
    print(x.shape)
    x = self.conv3(x)
    x = self.conv3_pool(x)
    print(x.shape)
    x = self.flatten(x)
    print(x.shape)
    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    print(x.shape)
    return x

model = TestCNN()
y = model(x)
print(y.shape)

(4, 14, 14, 10)
(4, 7, 7, 20)
(4, 3, 3, 30)
(4, 270)
(4, 10)
(4, 10)


Implementation with Sequential + Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Layer sub-classing 하는 법
# Layer를 상속 받음

self.conv1=Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding,
              activation='relu')
self.conv1_pool=MaxPooling2D(pool_size=pool_size, strides= pool_strides)

class MyConv(Layer): 
  def __init__(self, n_neuron):
    super(MyConv,self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding,
              activation='relu')
    self.conv_pool = MaxPooling2D(pool_size=pool_size, strides= pool_strides)

  def call(self,x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

# 즉 MyConv라는 클래스는 layer 한장을 만드는데, 그 layer는 conv, maxpooling 하나씩 있는 layer가 될것이다
# 그리고 후에 다른 곳에서 MyConv를 한번씩 호출하면 layer가 한 장씩 쌓이게 된다.
# 즉, layer를 내가 custom해서 묶음 단위로 가지고 다니는 것이다.

model = Sequential()
model.add(MyConv(n_conv_neurons[0])) # 이런 식으로 layer클래스 한 장씩 쌓아주는 것이다.
model.add(MyConv(n_conv_neurons[1])) 
model.add(MyConv(n_conv_neurons[2])) 

Implementation with Model + Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Layer sub-classing 하는 법
# Layer를 상속 받음

self.conv1=Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding,
              activation='relu')
self.conv1_pool=MaxPooling2D(pool_size=pool_size, strides= pool_strides)

class MyConv(Layer): 
  def __init__(self, n_neuron):
    super(MyConv,self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding,
              activation='relu')
    self.conv_pool = MaxPooling2D(pool_size=pool_size, strides= pool_strides)

  def call(self,x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN,self).__init__()

    global n_conv_neurons,n_dense_neurons

    self.conv1=Myconv(n_conv_neurons[0])
    self.conv2=Myconv(n_conv_neurons[1])
    self.conv3=Myconv(n_conv_neurons[2])

    self.flatten = Flatten()

    self.dense1= Dense(units=n_dense_neurons[0], activation='relu')
    self.dense2= Dense(units=n_dense_neurons[1], activation='relu')
    self.dense3= Dense(units=n_dense_neurons[2], activation='softmax')

  def call(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.flatten(x)

    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    return x



In [8]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class MyConv(Layer): 
  def __init__(self, n_neuron):
    super(MyConv,self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding,
              activation='relu')
    self.conv_pool = MaxPooling2D(pool_size=pool_size, strides= pool_strides)

  def call(self,x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN,self).__init__()

    global n_conv_neurons,n_dense_neurons
    
    self.fe = Sequential()
    self.fe.add(Myconv(n_conv_neurons[0]))
    self.fe.add(Myconv(n_conv_neurons[1]))
    self.fe.add(Myconv(n_conv_neurons[2]))
    self.fe.add(Flatten())

    self.classifier = Sequential()
    self.classifier.add(Dense(units=n_dense_neurons[0], activation='relu'))
    self.classifier.add(Dense(units=n_dense_neurons[1], activation='relu'))
    self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))
  def call(self,x):
    
    x = self.fe(x)
    x = self.classifier(x)

    return x
